# Inference without DS using CPU

I couldn't get Deepspeed to use the GPU for inference so for my research I opted to run inference on the CPU.
Your mileage may vary but this code has been included for completeness.

Find and replace "DATASET" with your dataset name.

In [ ]:
# install requirements
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html, https://download.pytorch.org/whl/torch_stable.html

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


## Setup

In [ ]:
# load libraries, model and tokenizer
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModelForCausalLM
import re
import torch

model = AutoModelForCausalLM.from_pretrained("./gpt-j-DATASET")

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

/home/jovyan/venvs/my_environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [ ]:
# Inference configuration parameters, see https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig 
# Exclusive parameters and output strategy is not clear from documentation, beam search seems to give best results


# When uncommenting parameters for use:
# Don't forget to uncomment the parameter to be used in every model.generate call below as well


num_returns = 3 # number of outputs per input prompt
# k = 100 # top-k
# p = 0.2 # top-p
temp = 0.8 # temperature
length = 140 # max additional number of tokens
beams = 5 # amount of beams
# n_gram = 2 # no repeat n-gram
# penalty = 0.5 # penalty alpha
# tpcl_p = 0.1 # typical p
# rep_penalty = 2.0 # repetition penalty
# eta_c = 0.0005 # eta cutoff

## Generation

One prompt from World of Warcraft is given as example input.

In [ ]:
# give input prompt of excluded original quests without description text so model generates from description tag onwards
prompt = "<|startoftext|>What Strange Creatures...<|level|>69<|race|>Horde<|category|>Shadowmoon Valley<|quest_giver|>Researcher Tiorus<|quest_ender|>Researcher Tiorus<|quest_line|>What Strange Creatures...<|required_quests|><|unlocked_quests|><|objective|>Researcher Tiorus at Shadowmoon Village in Shadowmoon Valley wants you to recover 8 Felfire Spleens.<|description|>"

# tokenize input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# generate tokens
gen_tokens = model.generate(input_ids,
                            do_sample=True, # whether or not to use sampling, False for greedy decoding
                            num_return_sequences=num_returns, 
                            # top_k=k,
                            # top_p=p,
                            temperature=temp,
                            max_new_tokens=length,
                            num_beams=beams,
                            # penalty_alpha=penalty,
                            # no_repeat_ngram_size=n_gram,
                            # typical_p=tpcl_p,
                            # repetition_penalty=rep_penalty,
                            # eta_cutoff=eta_c,
                            renormalize_logits=True
                            )

# print parameter settings
print("_" * 70 # separation line between entries
        + "\n"
      +", renormalize logits"
      # +", k: "+str(k)
      # +", p: "+str(p)
      +", temp: "+str(temp)
      +", length: "+str(length)
      +", beams: "+str(beams)
      # +", n-gram: "+str(n_gram)
      # +", penalty: "+str(penalty)
      # +", typical_p: "+str(tpcl_p)
    #   +", repetition penalty: "+str(rep_penalty)
      # +", ETA cutoff: "+str(eta_c)
      +"\n", file=open('DATASET_output.txt', 'a'))

# clean up results with regex and print result in DATASET_output.txt
for i, sample_output in enumerate(gen_tokens):
    print("{}: {}".format(i, re.sub(r'(\<\|)', r'\n\1', tokenizer.decode(sample_output, skip_special_tokens=True))), file=open('DATASET_output.txt', 'a'))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati